In [ ]:
#################################################################3
#EN ESTE NOTEBOOK USANDO YFINANCE OBTENGO MINUTO A MINUTO LOS DATOS DE 40 ACTIVOS, SEGUN DOCUMENTACION SE PUEDE SOLO 33 LLAMADAS POR MINUTO
#PERO SIEMPRE HAY EL RIESGO DEL ERROR 'TOO MANY REQUESTS', CON 20 ACTIVOS NO TUVE PROBLEMAS POR 4 DIAS SEGUIDOS CORRIENDO EL NOTEBOOK.
#EN LA ULTIMA SALIDA SE OBSERVA LOS VALORES DE CADA ACTIVO MINUTO A MINUTO, Y AÑADI UNOS CAMPOS A LOS OBTENIDOS:
# [REALTIME] 1|1|Acciones(S&P500)-Top10|  : POR EJEMPLO DESPUES DE 'REAL-TIME', EL PRIMER '1' REPRESENTA EL NUMERO DE ACTIVO ESTE NUMERO IRIA
# DE 1 A 40 EN CADA RONDA, EL SEGUNDO '1' REPRESENTA UN FLAG DE CAMBIO O NO EN BASE A LA FECHA/HORA DE COTIZACION, COMO SE SABE LA BOLSA Y OTROS
# ACTIVOS NO CAMBIAN POR MINUTO Y HAY DIAS EN QUE NO TRABAJAN ESOS ACTIVOS POR LO QUE LA FECHA/HORA DE LA COTIZACION PUEDE SER PASADA RESPECTO
# A LA FECHA DE SOLICITUD, POR ESO AL FINAL TAMBIEN AGREGUE UNA FECHA DE SOLICITUD DEL VALOR LUEGO DE LA FECHA DE COTIZACION REGISTRADA. EL FLAG:
# SERIA 1 SI AL SOLICITAR EL VALOR LA FECHA/HORA DE COTIZACION TAMBIEN SE ACTUALIZO CASO CONTRARIO FLAG=0 SERIA EN CASO NO HAYA CAMBIO
# ENTONCES SE VOLVERIA A REGISTRAR EL ULTIMO VALOR OBTENIDO DE LA RONDA ANTERIOR (MINUTO ANTERIOR) EN EL CASO DE ACCIONES COMO LA BOLSA DE VALORES
#NO TRABAJA FIN DE SEMANA LOS VALORES DEL SABADO Y DOMINGO SERIAN PRACTICAMENTE LOS DEL ULTIMO VIERNES.
#################################################################3

In [ ]:
#!pip install yfinance

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import os
import sys
import time
from typing import List, Dict, Set, Optional
from datetime import datetime, timedelta, timezone
import pandas as pd
from zoneinfo import ZoneInfo
import yfinance as yf

In [ ]:
# ====================================================================
# ========= Configuración y Tickers para Yfinance Multi-Activos =========
# ====================================================================

TZ_DEFAULT = "Europe/Madrid"

# Tasa de polling: Una vez por minuto (mantener para evitar rate limiting)
POLLING_INTERVAL_SEC = 60

# Diccionario de activos a monitorear (Total: 40 tickers)
ASSET_GROUPS: Dict[str, List[Dict[str, str]]] = {
    "Acciones(S&P500)-Top10": [
        {"symbol": "MSFT", "name": "Microsoft"},
        {"symbol": "AAPL", "name": "Apple"},
        {"symbol": "NVDA", "name": "NVIDIA"},
        {"symbol": "GOOGL", "name": "Alphabet (Google) Class A"},
        {"symbol": "AMZN", "name": "Amazon"},
        {"symbol": "META", "name": "Meta Platforms (Facebook)"}, # Nuevo
        {"symbol": "TSLA", "name": "Tesla"}, # Nuevo
        {"symbol": "JPM", "name": "JPMorgan Chase"}, # Nuevo
        {"symbol": "WMT", "name": "Walmart"}, # Nuevo
        {"symbol": "V", "name": "Visa"}, # Nuevo
    ],
    "Fondos(ETFs)-Principales": [
        {"symbol": "SPY", "name": "SPDR S&P 500 ETF Trust"},
        {"symbol": "QQQ", "name": "Invesco QQQ Trust"},
        {"symbol": "EEM", "name": "iShares MSCI Emerging Markets ETF"},
        {"symbol": "VGK", "name": "Vanguard FTSE Europe ETF"},
        {"symbol": "AGG", "name": "iShares Core U.S. Aggregate Bond ETF"},
    ],
    "Cripto(vsUSD)-Top10": [
        {"symbol": "BTC-USD", "name": "Bitcoin"},
        {"symbol": "ETH-USD", "name": "Ethereum"},
        {"symbol": "SOL-USD", "name": "Solana"},
        {"symbol": "XRP-USD", "name": "XRP"},
        {"symbol": "DOGE-USD", "name": "Dogecoin"},
        {"symbol": "ADA-USD", "name": "Cardano"}, # Nuevo
        {"symbol": "BNB-USD", "name": "BNB"}, # Nuevo
        {"symbol": "DOT-USD", "name": "Polkadot"}, # Nuevo
        {"symbol": "LINK-USD", "name": "Chainlink"}, # Nuevo
        {"symbol": "AVAX-USD", "name": "Avalanche"}, # Nuevo
    ],
    "Forex(TCambio)-Top10": [
        {"symbol": "EURUSD=X", "name": "Euro / Dólar"},
        {"symbol": "GBPUSD=X", "name": "Libra / Dólar"},
        {"symbol": "USDJPY=X", "name": "Dólar / Yen"},
        {"symbol": "USDCAD=X", "name": "Dólar / Dólar Canadiense"},
        {"symbol": "AUDUSD=X", "name": "Dólar Australiano / Dólar"},
        {"symbol": "USDCHF=X", "name": "Dólar / Franco Suizo"}, # Nuevo
        {"symbol": "EURGBP=X", "name": "Euro / Libra"}, # Nuevo
        {"symbol": "AUDJPY=X", "name": "Dólar Australiano / Yen"}, # Nuevo
        {"symbol": "NZDUSD=X", "name": "Dólar Neozelandés / Dólar"}, # Nuevo
        {"symbol": "EURJPY=X", "name": "Euro / Yen"}, # Nuevo
    ],
    "Commodities(Fut)-Top10": [
        {"symbol": "GC=F", "name": "Oro (Gold Futures)"}, #GLD
        {"symbol": "CL=F", "name": "Crudo (Crude Oil Futures)"}, #WTI
        {"symbol": "SI=F", "name": "Plata (Silver Futures)"}, #XAGUSD
        {"symbol": "PL=F", "name": "Platino (Platinum Futures)"}, #XPTUSD
        {"symbol": "ZC=F", "name": "Maíz (Corn Futures)"},#CORN
    ],
}

# Variable para rastrear el último precio/tiempo procesado por ticker
last_processed_price: Dict[str, float] = {}
last_processed_time: Dict[str, datetime] = {}


def log(msg: str): print(f"[INFO] {msg}", flush=True)
def warn(msg: str): print(f"[WARN] {msg}", flush=True)
def err(msg: str): print(f"[ERROR] {msg}", file=sys.stderr, flush=True)

# ========= Función de obtención de precio actual (Yfinance) =========
def fetch_current_price_yfinance(symbol: str) -> Optional[Dict]:
    """
    Obtiene el precio más reciente (retardo del mercado) y la hora de la cotización
    usando yfinance.
    """
    try:
        ticker = yf.Ticker(symbol)

        # Intentar obtener el precio de la información del ticker
        info = ticker.info
        price = info.get('regularMarketPrice') or info.get('currentPrice')
        time_raw = info.get('regularMarketTime')

        if price and time_raw:
            # Convertir el timestamp UNIX a datetime UTC
            dt_utc = datetime.fromtimestamp(time_raw, tz=timezone.utc)
            dt_local = dt_utc.astimezone(ZoneInfo(TZ_DEFAULT))
            return {"price": price, "time_local": dt_local}

        # Fallback: Intentar obtener la última vela de 1m del histórico
        data = ticker.history(period="1d", interval="1m", prepost=False)

        if not data.empty:
            latest_row = data.iloc[-1]
            price = latest_row['Close']

            # El índice es el tiempo de cierre de la vela (UTC)
            dt_utc = latest_row.name.tz_convert(timezone.utc)
            dt_local = dt_utc.astimezone(ZoneInfo(TZ_DEFAULT))

            return {"price": price, "time_local": dt_local}

        warn(f"{symbol}: No se encontraron datos válidos.")
        return None

    except Exception as e:
        err(f"{symbol}: Fallo al obtener datos con yfinance -> {e}")
        return None

# ========= Función de Polling en "Tiempo Real" Multi-Ticker =========
def real_time_polling_multi_ticker(symbols: List[str]):
    """
    Bucle de polling para múltiples tickers usando yfinance.
    Frecuencia: 1 llamada/ticker por minuto.
    """

    log(f"Inicializando rastreo de {len(symbols)} tickers...")
    # Inicializar diccionarios de rastreo
    local_tz = ZoneInfo(TZ_DEFAULT)
    # Inicializar el contador de rondas
    round_count = 0

    for symbol in symbols:
        last_processed_price[symbol] = 0.0
        last_processed_time[symbol] = datetime.min.replace(tzinfo=local_tz)

    try:
        while True:
            start_time = time.time()
            # Incrementar el contador de rondas al inicio de cada ciclo
            round_count += 1

            # --- NUEVAS VARIABLES DE RONDA ---
            # XXX3: Fecha/hora del sistema de la ronda
            round_system_time = datetime.now(ZoneInfo(TZ_DEFAULT)).strftime('%Y-%m-%d %H:%M:%S%z')
            # XXX1: Contador de ticket dentro de la ronda
            ticket_count = 0
            # ---------------------------------

            # Bucle interno: Ejecutar la ráfaga de llamadas
            for symbol in symbols:
                ticket_count += 1
                result = fetch_current_price_yfinance(symbol)

                if result is None:
                    warn(f"{symbol}: Fallo al obtener datos. Saltando en esta ronda.")
                    continue

                current_price = result["price"]
                current_time = result["time_local"]

                # # Solo mostrar si el precio ha cambiado o si el timestamp es más reciente
                # if current_price != last_processed_price[symbol] or current_time > last_processed_time[symbol]:
                #     # Determinar el grupo para una mejor visualización
                #     group_name = next((group for group, assets in ASSET_GROUPS.items() if any(a['symbol'] == symbol for a in assets)), "Otro")

                #     log(f"[REAL-TIME] [{group_name:25s}] {symbol:9s} | Precio: {current_price:,.4f} | Hora de Cotización (Local): {current_time:%H:%M:%S%z}")
                #     last_processed_price[symbol] = current_price
                #     last_processed_time[symbol] = current_time
                # else:
                #     # Mostrar que está esperando sin un log persistente
                #     print(f"[{current_time:%H:%M:%S%z}] {symbol}: Precio sin cambio ({current_price:,.4f}). Esperando nueva cotización...", end='\r')

               # --- CÁLCULO DE ESTADO DE CAMBIO (XXX2) ---
                group_name = next((group for group, assets in ASSET_GROUPS.items() if any(a['symbol'] == symbol for a in assets)), "Otro")

                # 1. Calcular el estado de cambio (boolean)
                # Usamos .get(symbol, 0.0) para la primera ronda
                has_changed = (current_price != last_processed_price.get(symbol, 0.0) or
                               current_time > last_processed_time.get(symbol, datetime.min.replace(tzinfo=local_tz)))

                # 2. Determinar el texto de cambio (XXX2)
                change_status_text = "1" if has_changed else "0"

                # 3. IMPRESIÓN DEL LOG OPTIMIZADA (sin formatos de ancho fijo)
                # Nuevo formato: [REAL-TIME] XXX1|XXX2|Grupo|Símbolo|Precio|Hora Cotización|Hora Solicitud XXX3
                log(f"[REALTIME] {ticket_count}|{change_status_text}|{group_name}|{symbol}|Precio: {current_price:,.4f}|Hora Coti(Local): {current_time:%H:%M:%S%z}|Hora Solicitud {round_system_time}")
                # 4. Actualizar el estado de rastreo
                last_processed_price[symbol] = current_price
                last_processed_time[symbol] = current_time
                # --- FIN CÁLCULO DE ESTADO DE CAMBIO ---

            # Control de tiempo para la siguiente ronda
            elapsed = time.time() - start_time
            sleep_time = POLLING_INTERVAL_SEC - elapsed

            if sleep_time > 0:
                try:
                    log(f"\nRonda completa {round_count} ({len(symbols)} llamadas). Esperando {sleep_time:.2f} segundos para la siguiente ronda.")
                    time.sleep(sleep_time)
                except KeyboardInterrupt:
                    raise KeyboardInterrupt

    except KeyboardInterrupt:
        log("Ejecución detenida por el usuario (Ctrl+C).")

In [ ]:
# ##########################################################################
#                       FUNCIÓN PRINCIPAL
# ##########################################################################

def main():
    try:
        import yfinance
    except ImportError:
        err("¡ERROR FATAL! La librería 'yfinance' no está instalada. Ejecuta: pip install yfinance")
        sys.exit(1)

    log(f"Zona horaria base: {TZ_DEFAULT}")

    # Aplanar todos los símbolos en una lista única
    all_symbols = [item['symbol'] for group in ASSET_GROUPS.values() for item in group]

    log(f"Total de Tickers seleccionados ({len(all_symbols)}): {', '.join(all_symbols)}")

   # 2. Iniciar el Polling Multi-Ticker
    log("\n#####################################################")
    log("### INICIANDO POLLING CON YFINANCE (Multi-Activos) ###")
    log(f"### Activos: {len(all_symbols)} Tickers. Frecuencia: 1 llamada/ticker cada {POLLING_INTERVAL_SEC} segundos. ###")
    log("### Pulsa Ctrl+C para DETENER. ###")
    log("#####################################################")

    try:
        real_time_polling_multi_ticker(all_symbols)
    except Exception as e:
        err(f"Fallo en el bucle principal de polling: {e}")

    log("✅ Listo.")

if __name__ == "__main__":
    main()

[INFO] Zona horaria base: Europe/Madrid
[INFO] Total de Tickers seleccionados (40): MSFT, AAPL, NVDA, GOOGL, AMZN, META, TSLA, JPM, WMT, V, SPY, QQQ, EEM, VGK, AGG, BTC-USD, ETH-USD, SOL-USD, XRP-USD, DOGE-USD, ADA-USD, BNB-USD, DOT-USD, LINK-USD, AVAX-USD, EURUSD=X, GBPUSD=X, USDJPY=X, USDCAD=X, AUDUSD=X, USDCHF=X, EURGBP=X, AUDJPY=X, NZDUSD=X, EURJPY=X, GC=F, CL=F, SI=F, PL=F, ZC=F
[INFO] 
#####################################################
[INFO] ### INICIANDO POLLING CON YFINANCE (Multi-Activos) ###
[INFO] ### Activos: 40 Tickers. Frecuencia: 1 llamada/ticker cada 60 segundos. ###
[INFO] ### Pulsa Ctrl+C para DETENER. ###
[INFO] #####################################################
[INFO] Inicializando rastreo de 40 tickers...
[INFO] [REALTIME] 1|1|Acciones(S&P500)-Top10|MSFT|Precio: 483.6200|Hora Coti(Local): 22:00:02+0100|Hora Solicitud 2026-01-02 05:44:35+0100
[INFO] [REALTIME] 2|1|Acciones(S&P500)-Top10|AAPL|Precio: 271.8600|Hora Coti(Local): 22:00:01+0100|Hora Solicitud 2026